In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from together import Together
from prompts.simpleqa_templates import GRADER_TEMPLATE, STUDENT_TEMPLATE, STUDENT_RAG_TEMPLATE
import argparse 
from tqdm import tqdm 
from sentence_transformers import SentenceTransformer, util, CrossEncoder

/Users/tanishqkumar/projects/home_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# both only 22m params
embedder = SentenceTransformer('all-MiniLM-L6-v2')
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

AttributeError: 'str' object has no attribute 'items'

In [11]:
device = torch.device("mps")
embedder.to(device); reranker.to(device)

CrossEncoder(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 384, padding_idx=0)
        (position_embeddings): Embedding(512, 384)
        (token_type_embeddings): Embedding(2, 384)
        (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-5): 6 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=384, out_features=384, bias=True)
                (key): Linear(in_features=384, out_features=384, bias=True)
                (value): Linear(in_features=384, out_features=384, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=384, out_features=384,

In [21]:
# Generate 200 questions and 4000 documents to test scalability
questions = ["In whose honor was the Leipzig 1877 tournament organized?"]
historical_figures = [
    "Napoleon Bonaparte", "Leonardo da Vinci", "Cleopatra VII",
    "Alexander the Great", "Adolf Anderssen", "Joan of Arc", "Julius Caesar", "Marie Curie",
    "Winston Churchill", "Genghis Khan"
]
documents = [historical_figures[i % len(historical_figures)] for i in range(4000)]

q_embs = embedder.encode(questions, convert_to_tensor=True, device=device) # [num_questions, d]
d_embs = embedder.encode(documents, convert_to_tensor=True, device=device) # [num_answers, d]
sims = torch.einsum('qd,ad->qa', q_embs, d_embs)
# this matrix is enough to get 
sims.shape # sims[0] is similarity over all answers for question 0

torch.Size([1, 4000])

In [23]:
topk_embed = 2
[documents[i] for i in torch.topk(sims[0], topk_embed).indices]

['Marie Curie', 'Marie Curie']

In [26]:
x = torch.randn(10, 10)
torch.topk(x, 2).indices

tensor([[9, 8],
        [0, 9],
        [7, 9],
        [9, 5],
        [6, 7],
        [9, 7],
        [5, 9],
        [7, 4],
        [2, 6],
        [1, 4]])